# Deep dive into Pandas Series data structure 

#### Read the official documentation on pandas Series objects @ https://pandas.pydata.org/pandas-docs/stable/reference/series.html

In [1]:
# import statements
import numpy as np
import pandas as pd

---------------------------
### Importing the data
------------------------

One of the datasets we will use for our examples in this notebook is the `/Data/vehicles.csv.zip` dataset. The first columns in the dataset we will investigate are `city08` and `highway08`, which provide information on miles per gallon usage while driving around in the city and highway respectively.

In [2]:
# read the vehicles.csv dataset
df = pd.read_csv("Data/vehicles.csv.zip")

# the city08 and highway08 columns from the vehicles.csv dataset
city_mpg = df.city08
highway_mpg = df.highway08

/tmp/ipykernel_7795/1072165566.py:2: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/vehicles.csv.zip")


---------------------------------------------
## Mathematical operations and Index Alignment 
------------------------------------------------

> The mathematical operations that are available include: **+, -, /, // (floor division), % (modulus), @ (matrix multiplication), ** (power), <, <=, ==, !=, >=, >, & (binary and), ^ (binary xor), | (binary or).**

However, pandas will **align the index** before performing any of these operations. Aligning will take each index entry in the left series and match it up with every entry with the same name in the index of the right series. Because of index alignment, you will want to **make sure that the indexes are, unique (no duplicates) and, common to both series**. Otherwise, the produced result will not be as expected.


- Example of how index alignment can cause to produce unintended results

In [3]:
s1 = pd.Series([10, 20, 30], index=[1, 2, 2])
s2 = pd.Series([15, 28, 32], index=[2, 2, 3])

In [4]:
s1 + s2

1     NaN
2    35.0
2    48.0
2    45.0
2    58.0
3     NaN
dtype: float64

Note that, index-1 and index-3 has NaN values whereas, for index-2, every index-2 value from s1 is matched with every index-2 value from s2. 

- Operator methods

Pandas also provides **operator methods** for each of the mathematical operations. The benefit is, the operator methods have a **fill_value** parameter that can change the default behavior. **By default, the operator methods will produce the same output as the mathematical operators themselves. But, if a fill_value is defined then when one of the operands is missing, the method will use the fill_value instead.**

> Some of the operator methods include: **add, sub, mul, div, mod, pow, rfloordiv, lt, gt, eq, ne, le, ge, dot, product** etc.

For instance, in the above example if we wanted to have values at index-1 and index-3 instead of NaN values we can use the **s1.add(s2, fill_value=0)** method. Probably, you can already guess how this will work out.

In [5]:
# using the add() method to add the elements of both series with fill_value=0
s1.add(s2, fill_value=0)

1    10.0
2    35.0
2    48.0
2    45.0
2    58.0
3    32.0
dtype: float64

Learn about all the available operator methods @ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#binary-operator-functions

---------------------------------------------
## Aggregate methods 
------------------------------------------------

Aggregate methods collapse the values of a series down to a scalar. Thus, allowing you to take detailed data and collapse it to a single value e.g, sum, count, mean, median etc.

> Some of the commonly used aggregate methods are, **all, any, count, prod, min, max, nsmallest, nlargest, cummax, cumsum, cumprod, mean, median, mode, sum, std, var, quantile, unique, nunique, value_counts, describe** etc.

- Counting non-null values

In [6]:
# Count total number of non-NA/null values in a series
print("no of non-NA vals:", city_mpg.count())

# but if we use count as an attribute then this will return a series containing the non-NA values

# city_mpg.count

no of non-NA vals: 41144


- Largest n elements

In [7]:
# return the largest n values
print("3 largest values: \n", city_mpg.nlargest(3))

3 largest values: 
 31256    150
32599    150
33423    150
Name: city08, dtype: int64


- Cumilitive properties

In [8]:
# cumilitive sum of a series
city_mpg.cumsum().tail()

41139    755704
41140    755724
41141    755742
41142    755760
41143    755776
Name: city08, dtype: int64

- Quantile values

Quantile is where probability distribution is divided into areas of equal probability. If we consider percentages, we first divide the distribution into 100 pieces. When we look into PDF, the 5th quantile is the point that cuts off an area of 5% in the lower tail of the distribution

In [9]:
# Quantile by default returns the 50% quantile. We can also pass in a list. In such case, this will return a series object
city_mpg.quantile([0.25, 0.5, 0.75])  # 25%, 50% and 75% quantiles

0.25    15.0
0.50    17.0
0.75    20.0
Name: city08, dtype: float64

- Generate descriptive statistics

In [10]:
city_mpg.describe()

count    41144.000000
mean        18.369045
std          7.905886
min          6.000000
25%         15.000000
50%         17.000000
75%         20.000000
max        150.000000
Name: city08, dtype: float64

- Unique values in a series

In [11]:
# return unique values in a series as ndarray
city_mpg.unique()

array([ 19,   9,  23,  10,  17,  21,  22,  18,  12,  20,  14,  11,  15,
        13,  16,  25,  24,  26,  31,  27,  30,  38,  28,  43,  35,  33,
        29,  39,  37,   8,   7,  34,  32,  36,  49,  81,  45,  48,  42,
         6,  44,  74,  84,  40,  87,  41,  51,  62,  59,  79,  50,  52,
       102, 106,  94, 126,  53, 107,  77, 110,  88, 132, 122, 138,  78,
        60,  47, 129,  93, 128,  61, 137,  85, 120,  86,  89,  95, 101,
        90, 124, 121,  54,  58,  91,  97,  73,  98,  92, 150,  55,  57,
        46, 118, 112, 131, 136,  83, 125,  80, 123, 127, 114, 140, 115,
       104])

In [12]:
# return a series containing counts of unique values
city_mpg.value_counts(dropna=False)  # by default, dropna=True

15     4503
18     4053
17     4035
16     3975
19     3012
       ... 
127       1
114       1
140       1
115       1
104       1
Name: city08, Length: 105, dtype: int64

### The _.agg()_ function 

The `.agg()` function can be used to perform multiple aggregate operations (you can also define your own aggregate functions) on a series object at the same time.

In [13]:
city_mpg.agg(
    ["min", "idxmin", "max", "idxmax", "mean", "std", "var", "quantile", "all", "sum"]
)

min                 6
idxmin           7901
max               150
idxmax          31256
mean        18.369045
std          7.905886
var         62.503036
quantile         17.0
all              True
sum            755776
Name: city08, dtype: object

To learn about all the available aggregate functions see the documentation @ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#computations-descriptive-stats